In [219]:
import sys
import json
import argparse
import random
import csv
import numpy as np
from scipy.signal import argrelextrema
from pandas import read_csv
import random


import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization

In [253]:

def read_file(file_path, type):
    if type == "csv":
        data = read_csv(file_path)
        return data.values
    elif type == "json":
        f = open(file_path)
        data = json.load(f)
        return data

In [354]:
session_path = 'toadstool/participants/participant_9/participant_9_session.json'
truth_path = 'truth/participant_9_events.json'
vtruth_path = 'vtruth/participant_9_video.csv'

In [355]:
 # find local maxima index
    random_para = 10
    # event we have
    event = {0:"new_stage", 1:"flag_reached", 2:"status_up", 3:"status_down", 4:"life_lost"}
    out = {"new_stage":0, "flag_reached":1, "status_up":2, "status_down":3, "life_lost":4}
 # game frame number
    game = read_file(session_path, "json")#frame的每個數字
    game_len = int(game["obs_n"])
    
    bvp = read_file(bvp_path, "csv")
    local_max_idx = argrelextrema(bvp, np.greater)
#     target = pd.DataFrame( local_max_idx)

    result = []
    for i in local_max_idx[0]:
        event_idx = random.randint(0, random_para)
        if event_idx < len(event):
            event_name = event[event_idx]
            frame_number = i
            row = [frame_number, event_name]
            result.append(row)

In [356]:
game_len

125381

In [357]:
 #train ground truth

    truth = read_file(truth_path, "json")
    truth = pd.DataFrame(truth)
    truth

,event,frame_number
0,life_lost,224
1,status_up,658
2,status_up,1636
3,flag_reached,2956
4,new_stage,2957
...,...,...
156,life_lost,120975
157,life_lost,121697
158,life_lost,122751
159,life_lost,123215


In [358]:
#emotional truth
    vtruth = read_file(vtruth_path, "csv")
    emotion = pd.DataFrame(vtruth)
    emotion = emotion.drop([1],axis=1)
    emotion.shape

(62958, 8)

In [359]:
emotion.shape[0]

62958

In [360]:
frame = list(range(emotion.shape[0]))

In [361]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
import numpy as ny
newframe = []
frame = list(range(emotion.shape[0]))
frame=np.array(frame)
frame = frame[:,np.newaxis]

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(frame)

for i in scaled:
    i=float(i*game_len)
    newframe.append(round(i))

In [362]:
scaled

array([[0.00000000e+00],
       [1.58838572e-05],
       [3.17677145e-05],
       ...,
       [9.99968232e-01],
       [9.99984116e-01],
       [1.00000000e+00]])

In [363]:
emotion['normalize'] = newframe
emotion.set_index(emotion['normalize'],inplace=True)
emotion = emotion.drop(['normalize'],axis=1)
#drop 2,3,7,8 angry,disgust,surprise,neutral,box
# emotion = emotion.drop([2],axis=1)
emotion = emotion.drop([3],axis=1)
# emotion = emotion.drop([7],axis=1)
emotion = emotion.drop([8],axis=1)
emotion = emotion.drop([0],axis=1)
emotion.columns = ['angry','fear','happy','sad','surprise']
emotion

,angry,fear,happy,sad,surprise
normalize,,,,,
0,0.07,0.16,0,0.08,0.02
2,0.04,0.17,0.01,0.3,0.01
4,0.08,0.2,0,0.18,0.02
6,0.09,0.23,0,0.13,0.02
8,0.08,0.3,0,0.15,0.01
...,...,...,...,...,...
125373,0.03,0.1,0.64,0.02,0.19
125375,0,0.01,0.95,0,0.03
125377,0,0.03,0.89,0,0.07


In [364]:
def findnear(arr,val):
    arr = np.array(arr)
    idx = (np.abs(arr-val)).argmin()
    return arr[idx]

In [365]:
farr = np.array(newframe)

In [366]:
A= np.array(truth['frame_number'])
A

array([   224,    658,   1636,   2956,   2957,   3226,   3642,   3928,
         4020,   5514,   7062,  10125,  10126,  12185,  12839,  15141,
        15142,  17428,  17429,  17743,  19579,  19580,  21566,  22614,
        24418,  24848,  25242,  25650,  26861,  28631,  30245,  32197,
        32647,  32837,  33247,  34505,  34917,  35801,  36815,  38479,
        38705,  39971,  41925,  41925,  42191,  42703,  44190,  44590,
        45056,  46786,  47082,  47296,  47604,  47864,  48060,  48314,
        48502,  49096,  49790,  50068,  50786,  51054,  51380,  52396,
        52772,  52972,  53204,  54798,  55435,  55436,  56506,  57220,
        57922,  59966,  59967,  60678,  61106,  61329,  61328,  61754,
        63654,  64170,  64552,  65460,  65459,  65991,  67105,  67873,
        68225,  69372,  70489,  70490,  71595,  71987,  73655,  73656,
        74779,  75747,  76669,  76670,  77253,  77350,  77349,  78005,
        78232,  79895,  79896,  80351,  81071,  81663,  82079,  82442,
      

In [367]:
emolist=[]
check=[]
for i in A:#選最近數字
        nearest = findnear(farr,i)
        arr = np.array(emotion.loc[nearest])
        emolist.append(arr)
        check.append(nearest)#427

In [368]:
farr

array([     0,      2,      4, ..., 125377, 125379, 125381])

In [369]:
emodf = pd.DataFrame(emolist)
emodf.columns = ['angry','fear','happy','sad','surprise']
emodf

,angry,fear,happy,sad,surprise
0,0.10,0.34,0.00,0.16,0.02
1,0.23,0.32,0.00,0.18,0.06
2,0.12,0.13,0.02,0.16,0.02
3,0.28,0.22,0.00,0.27,0.02
4,0.24,0.18,0.00,0.31,0.04
...,...,...,...,...,...
156,0.22,0.07,0.00,0.43,0.01
157,0.13,0.08,0.00,0.19,0.03
158,0.10,0.31,0.00,0.17,0.05
159,0.02,0.17,0.00,0.17,0.07


In [370]:
#findmax and record
#這個是要跟圖片一起的數字
emomax = np.array(emodf)
e = []
for i in emomax:
    e.append(np.argmax(i))

In [371]:
emodf['trueframe'] = check
emodf['emomax'] = e
emodf

,angry,fear,happy,sad,surprise,trueframe,emomax
0,0.10,0.34,0.00,0.16,0.02,223,1
1,0.23,0.32,0.00,0.18,0.06,657,1
2,0.12,0.13,0.02,0.16,0.02,1635,3
3,0.28,0.22,0.00,0.27,0.02,2955,0
4,0.24,0.18,0.00,0.31,0.04,2957,3
...,...,...,...,...,...,...,...
156,0.22,0.07,0.00,0.43,0.01,120974,3
157,0.13,0.08,0.00,0.19,0.03,121697,3
158,0.10,0.31,0.00,0.17,0.05,122750,1
159,0.02,0.17,0.00,0.17,0.07,123214,1


In [372]:
count=list(range(emotion.shape[0]))
emotion['cnt']=count
emotion

,angry,fear,happy,sad,surprise,cnt
normalize,,,,,,
0,0.07,0.16,0,0.08,0.02,0
2,0.04,0.17,0.01,0.3,0.01,1
4,0.08,0.2,0,0.18,0.02,2
6,0.09,0.23,0,0.13,0.02,3
8,0.08,0.3,0,0.15,0.01,4
...,...,...,...,...,...,...
125373,0.03,0.1,0.64,0.02,0.19,62953
125375,0,0.01,0.95,0,0.03,62954
125377,0,0.03,0.89,0,0.07,62955


In [373]:
idx=[]
for c in check:
    idx.append(emotion.at[c,'cnt'])


In [374]:
len(idx)

161

In [375]:
chooseframe=[]
for i in range(len(emotion)):
    if i in idx:
        chooseframe.append(1)
    else:
        chooseframe.append(0)
emotion['chooseframe'] = chooseframe  
len(chooseframe)

62958

In [376]:
emotion.at[check[0],'chooseframe']

1

In [377]:
    # write output
    output_path='emotrain/e9.csv'
    emotion.to_csv(output_path, index = True)

In [245]:
ey_train = emotion['chooseframe']
ex_train =emotion.drop(['cnt'],axis=1)

In [246]:
j=0

In [247]:
for i in ey_train:
    if i==1:
        j=j+1
print(j)

117


In [248]:
len(check)

123

In [159]:
ex_train

,angry,fear,happy,sad,surprise,chooseframe
normalize,,,,,,
0,0.09,0.06,0,0.38,0,0
2,0.11,0.06,0,0.32,0,0
4,0.08,0.03,0,0.29,0,0
6,0.08,0.03,0,0.31,0,0
8,0.08,0.03,0,0.45,0,0
...,...,...,...,...,...,...
125415,0.07,0.1,0.41,0.14,0.01,0
125417,0.12,0.09,0.26,0.19,0,0
125419,0.09,0.07,0.19,0.26,0.01,0


In [160]:
ey_train

normalize
0         0
2         0
4         0
6         0
8         0
         ..
125415    0
125417    0
125419    0
125421    0
125423    0
Name: chooseframe, Length: 62831, dtype: int64

In [256]:
#Split the data into training and validation sets
#Build you model and evaluate it on your validation set
from sklearn.model_selection import train_test_split

ex =ex_train
ey =ey_train
ex_train, ex_test, ey_train, ey_test = train_test_split(ex,ey,test_size=0.3,random_state=0)

In [258]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(ex_train,ey_train)

RandomForestClassifier()

In [259]:
rf_pred = rf.predict(ex_test)

In [260]:
rf_pred

array([0, 0, 0, ..., 0, 0, 0])

In [261]:
import sklearn.metrics as metrics
print("Accuracy:",metrics.accuracy_score(ey_test, rf_pred))

Accuracy: 1.0


In [379]:
import os

yourPath = 'scripts/i1'
allFileList = os.listdir(yourPath)
X_train = []
Y_train = []
T= list(truth['frame_number'])
P=[]

for i in T:
    for file in allFileList:
        if file=='game_'+str(i)+'.png':
            p = yourPath + "/" +file
#             img = cv2.imread(p)
            P.append(p) 
truth['img'] = P
truth

,event,frame_number,img
0,life_lost,224,scripts/i1/game_224.png
1,status_up,658,scripts/i1/game_658.png
2,status_up,1636,scripts/i1/game_1636.png
3,flag_reached,2956,scripts/i1/game_2956.png
4,new_stage,2957,scripts/i1/game_2957.png
...,...,...,...
156,life_lost,120975,scripts/i1/game_120975.png
157,life_lost,121697,scripts/i1/game_121697.png
158,life_lost,122751,scripts/i1/game_122751.png
159,life_lost,123215,scripts/i1/game_123215.png


In [76]:
train = pd.concat([truth, emodf], axis = 1)
line = []
for i in train['event']:
    line.append(out[i])
    
train['out'] = line
train

,event,frame_number,img,angry,fear,happy,sad,surprise,trueframe,emomax,out
0,status_up,428,scripts/i0/game_428.png,0.01,0.04,0.01,0.03,0.00,427,1,2
1,flag_reached,1915,scripts/i0/game_1915.png,0.02,0.12,0.00,0.06,0.01,1914,1,1
2,new_stage,1916,scripts/i0/game_1916.png,0.01,0.10,0.00,0.05,0.00,1916,1,0
3,life_lost,2003,scripts/i0/game_2003.png,0.02,0.17,0.00,0.09,0.02,2003,1,4
4,status_up,2369,scripts/i0/game_2369.png,0.01,0.06,0.00,0.06,0.00,2369,1,2
...,...,...,...,...,...,...,...,...,...,...,...
164,new_stage,109535,scripts/i0/game_109535.png,0.02,0.09,0.00,0.06,0.00,109534,1,0
165,life_lost,117553,scripts/i0/game_117553.png,0.01,0.06,0.04,0.05,0.00,117553,1,4
166,life_lost,121735,scripts/i0/game_121735.png,0.02,0.03,0.21,0.03,0.00,121735,2,4
167,life_lost,122504,scripts/i0/game_122504.png,0.03,0.06,0.01,0.11,0.00,122504,3,4


In [199]:
import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization

In [183]:
itrain = pd.concat([train['out'], train['img'],train['emomax']], axis = 1)
itrain

,out,img,emomax
0,2,scripts/i0/game_428.png,1
1,1,scripts/i0/game_1915.png,1
2,0,scripts/i0/game_1916.png,1
3,4,scripts/i0/game_2003.png,1
4,2,scripts/i0/game_2369.png,1
...,...,...,...
164,0,scripts/i0/game_109535.png,1
165,4,scripts/i0/game_117553.png,1
166,4,scripts/i0/game_121735.png,2
167,4,scripts/i0/game_122504.png,3


In [116]:
train_images.shape

(169, 128, 128, 3)

In [143]:
X = itrain['img']
image_list = []
for file_name in X:
    image_array = np.array(PIL.Image.open(file_name))
    resized_image_array = cv2.resize(image_array, (128, 128))
    image_list.append(resized_image_array)
train_images = np.array(image_list)

In [144]:
earr=np.array(itrain['emomax'])
earr=np.reshape(earr,(1,169,1,1))
earr

array([[[[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[3]],

        [[3]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[3]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        [[1]],

        

In [145]:
plus=np.insert(train_images,0,values=earr,axis=1)

In [146]:
plus.shape

(169, 129, 128, 3)

In [235]:
y = itrain['out']
X =plus
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_train = y_train.to_numpy(dtype = int)
y_test = y_test.to_numpy(dtype = int)

In [189]:
y.shape

(169,)

In [190]:
y_test.shape

(51,)

In [191]:
X_test.shape

(51, 129, 128, 3)

In [192]:
y_train = y_train.reshape((-1,1))

In [193]:
model = Sequential()
model.add(Conv2D(input_shape=(129, 128, 3), filters=96, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=5, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 129, 128, 96)      4704      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 96)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 64, 96)        384       
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 64, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       393472    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 256)      

In [194]:
import tensorflow as tf
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [195]:
y_train.shape

(118, 1)

In [49]:
model = keras.models.load_model('mario.h5')

In [196]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) # Callback for earlystopping
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy" , metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/15
4/4 [==============================] - 24s 6s/step - loss: 6.1609 - accuracy: 0.4153 - val_loss: 26.7207 - val_accuracy: 0.6275
Epoch 2/15
4/4 [==============================] - 17s 4s/step - loss: 2.4948 - accuracy: 0.6610 - val_loss: 16.1362 - val_accuracy: 0.2745
Epoch 3/15
4/4 [==============================] - 19s 5s/step - loss: 1.1200 - accuracy: 0.8475 - val_loss: 6.1609 - val_accuracy: 0.7451
Epoch 4/15
4/4 [==============================] - 20s 5s/step - loss: 0.3179 - accuracy: 0.9492 - val_loss: 5.7179 - val_accuracy: 0.7255
Epoch 5/15
4/4 [==============================] - 19s 5s/step - loss: 0.1374 - accuracy: 0.9661 - val_loss: 4.5975 - val_accuracy: 0.7647
Epoch 6/15
4/4 [==============================] - 20s 5s/step - loss: 0.1669 - accuracy: 0.9576 - val_loss: 4.4762 - val_accuracy: 0.7255
Epoch 7/15
4/4 [==============================] - 18s 5s/step - loss: 0.1048 - accuracy: 0.9661 - val_loss: 3.9847 - val_accuracy: 0.7451
Epoch 8/15
4/4 [================

In [221]:
model.save('mario.h5')

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(test_acc)

2/2 - 2s - loss: 2.4859 - accuracy: 0.7451
0.7450980544090271


In [231]:
prediction = model.predict(X)

In [233]:
prediction = np.array(prediction)
result = []
for i in prediction:
    result.append(np.argmax(i))
len(result)

169

In [236]:
import sklearn
from sklearn.metrics import f1_score

score = f1_score(y, result, average=None)

print(score)

[0.81481481 0.8627451  0.         0.         0.87671233]


In [262]:
re=[]
for i in result:
    re.append(event[i])
train['result']=re
train

,event,frame_number,img,angry,fear,happy,sad,surprise,trueframe,emomax,out,result
0,status_up,428,scripts/i0/game_428.png,0.01,0.04,0.01,0.03,0.00,427,1,2,life_lost
1,flag_reached,1915,scripts/i0/game_1915.png,0.02,0.12,0.00,0.06,0.01,1914,1,1,flag_reached
2,new_stage,1916,scripts/i0/game_1916.png,0.01,0.10,0.00,0.05,0.00,1916,1,0,life_lost
3,life_lost,2003,scripts/i0/game_2003.png,0.02,0.17,0.00,0.09,0.02,2003,1,4,life_lost
4,status_up,2369,scripts/i0/game_2369.png,0.01,0.06,0.00,0.06,0.00,2369,1,2,life_lost
...,...,...,...,...,...,...,...,...,...,...,...,...
164,new_stage,109535,scripts/i0/game_109535.png,0.02,0.09,0.00,0.06,0.00,109534,1,0,new_stage
165,life_lost,117553,scripts/i0/game_117553.png,0.01,0.06,0.04,0.05,0.00,117553,1,4,life_lost
166,life_lost,121735,scripts/i0/game_121735.png,0.02,0.03,0.21,0.03,0.00,121735,2,4,life_lost
167,life_lost,122504,scripts/i0/game_122504.png,0.03,0.06,0.01,0.11,0.00,122504,3,4,life_lost


In [277]:
final = pd.concat([train['trueframe'], train['result']], axis = 1)
final

,trueframe,result
0,427,life_lost
1,1914,flag_reached
2,1916,life_lost
3,2003,life_lost
4,2369,life_lost
...,...,...
164,109534,new_stage
165,117553,life_lost
166,121735,life_lost
167,122504,life_lost


In [278]:
    # write output
    output_path='run/participant_0_events.csv'
    final.to_csv(output_path, index = False,header=False)